In [60]:
import sys
import numpy as np
import pandas as pd
import pysal as ps
sys.path.append('/Users/toshan/Dropbox/GWR/PyGWRJing/PyGWR/')
from M_FBGWR_May2016 import FBGWR
from M_GWGLM import GWGLM
from M_selection import Band_Sel
import scipy

In [61]:
path = ps.examples.get_path('GData_utm.csv')
shp = pd.read_csv(path)

In [62]:
# Prep data into design matrix and coordinates

#Dependent variable
y = shp.PctBach.reshape((-1,1))

#Design matrix - covariates - intercept added automatically
pov = shp.PctPov.reshape((-1,1))
rural = shp.PctRural.reshape((-1,1))
blk = shp.PctBlack.reshape((-1,1))
X = np.hstack([rural, pov, blk])
labels = ['Intercept', 'PctRural', 'PctPov', 'PctBlack']

#Coordinates for calibration points
u = shp.X
v = shp.Y
coords = zip(u,v)

coords_dict = {}
for i, x in enumerate(coords):
        coords_dict[i] = x

In [63]:
%%time
print Band_Sel(y, None, X, coords_dict)

(60.0, <Kernel.GWR_W object at 0x10c54b690>, [(89.0, 1088.4132720309442), (73.0, 1082.7676645259439), (62.0, 1079.8414440181944), (62.0, 1079.8414440181944), (62.0, 1079.8414440181944), (60.0, 1079.666817791637), (60.0, 1079.666817791637), (60.0, 1079.666817791637)])
CPU times: user 1.84 s, sys: 74 ms, total: 1.91 s
Wall time: 1.85 s


In [70]:
%%time
results1 = FBGWR(y, X, coords_dict, tolFB=1e-03)

0.0655846762588
0.0233486761322
0.0163002322842
0.00433277272955
0.00954961334306
0.00290595644909
0.00195096730304
0.00160683793335
0.00134567834122
0.00113425557505
0.00153002056533
CPU times: user 1min, sys: 187 ms, total: 1min
Wall time: 1min


In [71]:
from pysal.contrib.gwr.sel_bw import Sel_BW
from pysal.contrib.gwr.gwr import GWR, FBGWR

In [72]:
%%time
print Sel_BW(coords, y, X, [], kernel='bisquare', constant=False).search()

60.0
CPU times: user 1.12 s, sys: 2.96 ms, total: 1.12 s
Wall time: 1.12 s


In [73]:
%%time
sel = Sel_BW(coords, y, X, [], kernel='bisquare', fb=True, constant=False)
results = sel.search(tol_fb=1e-03)

0.0221239134234
0.0151919208769
0.00862867394367
0.00458947306683
0.0035545499035
0.00279186902711
0.00211662633452
0.00162432236
0.00126749982983
0.00101411591791
0.00088200956837
CPU times: user 37.5 s, sys: 89.9 ms, total: 37.6 s
Wall time: 37.5 s


In [74]:
print results
XB = sel.XB
err = sel.err
print sel.XB.shape

[157.0, 65.0, 52.0]
(159, 3)


In [92]:
model = FBGWR(coords, y, X, [157.0, 65.0, 52.0], XB, err, constant=False)

In [93]:
results  = model.fit()

In [123]:
np.allclose(results.predy, results1.y_pred, atol=1e-08)

True

In [124]:
np.allclose(results.params, results1.Betas, atol=1e-08)

True

In [107]:
np.allclose(results.u, results1.y-results1.y_pred, atol=1e-06)

True

In [111]:
np.allclose(results.utu, np.dot(results1.y-results1.y_pred, (results1.y-results1.y_pred).T), atol=1e-05)

True

In [81]:
rslt = {'predy':results1.y_pred, 'params':results1.Betas, 'u':results1.y-results1.y_pred, 'utu': np.dot(results1.y-results1.y_pred, (results1.y-results1.y_pred).T)}

In [112]:
import pickle as pk

In [113]:
pk.dump(rslt, open('/Users/toshan/dev/pysal/pysal/examples/georgia/FB.p', 'w'))

In [114]:
pk.dump(XB, open('/Users/toshan/dev/pysal/pysal/examples/georgia/XB.p', 'w'))

In [115]:
pk.dump(err, open('/Users/toshan/dev/pysal/pysal/examples/georgia/err.p', 'w'))

In [117]:
results.predy

array([[  7.64046455],
       [  9.33041206],
       [  7.4838787 ],
       [ 10.35169757],
       [  7.47191829],
       [ 11.29155827],
       [ 12.92109341],
       [ 11.68389652],
       [  8.34745367],
       [  5.57766735],
       [  9.26363503],
       [  6.89861011],
       [  5.84888999],
       [ 10.69363   ],
       [  6.34092966],
       [ 10.20826899],
       [ 10.82648501],
       [ 12.98015799],
       [ 11.64497376],
       [  6.46643517],
       [  8.85282233],
       [ 10.90017107],
       [  7.61823717],
       [  9.16379157],
       [  9.79531986],
       [  5.02553059],
       [ 12.44481148],
       [  5.62517436],
       [ 16.44650578],
       [ 12.05978295],
       [ 10.82167033],
       [  9.43203161],
       [  7.62722642],
       [  7.9473997 ],
       [  7.29552045],
       [  2.69637503],
       [  8.54894648],
       [ 10.62310719],
       [  8.0035713 ],
       [ 10.06150168],
       [  9.21841152],
       [ 10.05231168],
       [  8.44785642],
       [ 21

In [99]:
results1.y-results1.y_pred

array([[  5.59535569e-01],
       [ -2.93041181e+00],
       [ -8.83878499e-01],
       [ -9.51697600e-01],
       [  5.82808197e+00],
       [ -4.89155881e+00],
       [ -3.72109434e+00],
       [ -2.68389670e+00],
       [ -7.47453620e-01],
       [  1.92233285e+00],
       [  7.73636453e+00],
       [  3.40138972e+00],
       [ -4.88898147e-02],
       [ -1.59363035e+00],
       [  5.45907022e+00],
       [  9.69173075e+00],
       [ -1.22648481e+00],
       [ -5.78015834e+00],
       [ -1.54497362e+00],
       [  7.03356463e+00],
       [  1.04717754e+00],
       [  1.09982889e+00],
       [  4.81762966e-01],
       [ -2.76379156e+00],
       [  8.80467952e+00],
       [  1.51744693e+01],
       [ -6.54481146e+00],
       [  1.27748256e+01],
       [  2.10534937e+01],
       [ -8.59782689e-01],
       [  3.87832912e+00],
       [ -2.73203152e+00],
       [  2.53727731e+01],
       [  3.15260048e+00],
       [  2.70447953e+00],
       [  2.12036250e+01],
       [ -2.04894662e+00],
 